# Modern Portfolio Theory

## 1. Libraries

In [1]:
# Market data from https://www.quandl.com
import pandas as pd  
import numpy as np
import quandl
import scipy.optimize as sco

## 2. Working with data

### 2.1. Quandl API

In [2]:
quandl.ApiConfig.api_key = open('Quandl_key.txt').readline()
stocks = ['AAPL','AMZN','GOOGL','FB']
data = quandl.get_table('WIKI/PRICES', ticker = stocks,
                        qopts = { 'columns': ['date', 'ticker', 'adj_close'] },
                        date = { 'gte': '2017-1-1', 'lte': '2017-31-12' }, paginate=True)

df = data.set_index('date')
table = df.pivot(columns='ticker')
table.columns = [col[1] for col in table.columns]

table.head()

,AAPL,AMZN,FB,GOOGL
date,,,,
2017-01-03,114.715378,753.67,116.86,808.01
2017-01-04,114.586983,757.18,118.69,807.77
2017-01-05,115.169696,780.45,120.67,813.02
2017-01-06,116.453639,795.99,123.41,825.21
2017-01-09,117.520300,796.92,124.90,827.18


### 2.2. Checking data

In [3]:
table.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 250 entries, 2017-01-03 to 2017-12-29
Data columns (total 4 columns):
AAPL     249 non-null float64
AMZN     249 non-null float64
FB       250 non-null float64
GOOGL    250 non-null float64
dtypes: float64(4)
memory usage: 9.8 KB


In [4]:
table[table['AAPL'].isnull() == True]

,AAPL,AMZN,FB,GOOGL
date,,,,
2017-08-07,NaN,NaN,171.98,945.75


As we can see, we have 1 row with null values for AAPL and AMZN. I will fill mising value by value from previous trading day

In [5]:
table['AAPL'].iloc[table.index.get_loc('2017-08-07')] = table['AAPL'].iloc[table.index.get_loc('2017-08-07') - 1]
table['AMZN'].iloc[table.index.get_loc('2017-08-07')] = table['AMZN'].iloc[table.index.get_loc('2017-08-07') - 1]

In [6]:
table.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 250 entries, 2017-01-03 to 2017-12-29
Data columns (total 4 columns):
AAPL     250 non-null float64
AMZN     250 non-null float64
FB       250 non-null float64
GOOGL    250 non-null float64
dtypes: float64(4)
memory usage: 19.8 KB


## 3. Implementing MPT

In [7]:
returns = table.pct_change()
mean_returns = returns.mean()
cov_matrix = returns.cov()
risk_free_rate = 0.0172
#https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield

In [8]:
def portfolio_annualised_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns*weights )*250
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(250)
    return std, returns

In [9]:
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    p_var, p_ret = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
    return -(p_ret - risk_free_rate) / p_var

In [10]:
def max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(neg_sharpe_ratio, num_assets*[1./num_assets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [11]:
max_sharpe = max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
allocations = np.round(max_sharpe['x']*100,2)
allocations

array([35.8 , 17.09, 47.12,  0.  ])

## 4. Portfolio

In [12]:
allocations

array([35.8 , 17.09, 47.12,  0.  ])

In [13]:
portfolio_std, portfolio_return = portfolio_annualised_performance(allocations, mean_returns, cov_matrix)
portfolio_std, portfolio_return

(15.192919255905686, 42.6374490346388)

In [14]:
sharpe_ratio = -neg_sharpe_ratio(allocations, mean_returns, cov_matrix, risk_free_rate)
sharpe_ratio

2.8052705551022887